<a href="https://colab.research.google.com/github/Imran-Github22/Programming-For-Data/blob/main/project/12_WatsonToneAnalyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini-project - creating a dataframe from analysed text data

For this project you are going to use the IBM Watson Tone Analyser API.  You will send text data to it, use security information stored in a config file to keep it secret, receive the results in JSON format, investigate the structure of the results and build a dataframe from them.

Then you will use the results to create a visualisation of tone and to report an overall set of statistics from the data.

---

## Step 1 - sign up for IBM Watson services to use the Tone Analyser

1.  Sign up for [IBM Watson](https://cloud.ibm.com/registration?cm_mmc=dw-_-cognitive-_-topcoder-_-communityEducational1)
2.  Click 'Try on Cloud at no cost'  
3.  Select the London region  (costs reduced and performance improved when you use the nearest servers)  
4.  Create an IBM Cloud account (enter email and accept terms)  
5.  Follow the instructions to create the account  
6.  Provision the services  
7.  Then go to IBM Watson Studio  
8.  Select Tone Analyzer under the Your Services heading  
9.  You will be shown the **url** for the Tone Analyser API and an **API key** which is needed for using the API.

# Step 2 - add security to your worksheet to keep your apikey and url hidden

You can do this by using environment variables, which are stored in the operating system for this worksheet.

We will use a simplified system for storing the sensitive data so that it isn't visible in the worksheet:

1.  Ask for the api key to be input and store it in an environment variable called apikey

2.  Ask for the url to be input and store it in an environment variable

3.  Run the cell, type in the api key, then the url.  Once tis has been done.  Remove the output part of the cell.

In [ ]:
import os

# add the code to ask for the URL, then run this cell and when it has completed, remove the output (note: you will need to do this again if you return to the worksheet)
os.environ['APIKEY'] = input("Enter API key: ")
os.environ['URL'] = input("Enter URL: ")


# Install the ibm_watson libraries so that you can use their functions

In [ ]:
# install the ibm_watson libraries

!pip install ibm_watson

---

## Test to make sure it works

1.  Open this file, which has some text for you to test with: https://drive.google.com/file/d/1m65cPQGYQd1mwvEmfZw69-GMUBdo43k0/view?usp=sharing.  You will be able to copy and paste the text into here as needed.

2.  Get the environment variable for each of the two pieces of security information so that these do not need to be included in your notebook (have the keys available for copying and pasting).  To do this:

  ``` apikey = os.environ.get('APIKEY') ```

3.  Run the code below,which will create a ToneAnalyzer with the credentials from your environment variables, then paste the text from the **text-for-analysis.txt** file

4.  Decide what the data looks like and how this might be represented in a pandas dataframe

In [ ]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import os
import json

# get credentials from the environment variables you set
def get_secret(key):
    # add code here to get the keys from the environment variable and return the requested key
    # if there is an error print an error message and return None
    return os.environ.get(key)


    
def get_text_for_analysis():
    # add code here to input the text from the text-for-analysis.txt file and return the text it reads as one string
    # if there is an error, return None
    # return 'Team, I know that times are tough! Product sales have been disappointing for the past three quarters. We have a competitive product, but we need to do a better job of selling it!'
    return 'But I feel peaceful. Your success in the ring this morning was, to a small degree, my success. Your future is assured. You will live, secure and safe, Wilbur. Nothing can harm you now. These autumn days will shorten and grow cold. The leaves will shake loose from the trees and fall. Christmas will come, and the snows of winter. You will live to enjoy the beauty of the frozen world, for you mean a great deal to Zuckerman and he will not harm you, ever. Winter will pass, the days will lengthen, the ice will melt in the pasture pond. The song sparrow will return and sing, the frogs will awake, the warm wind will blow again. All these sights and sounds and smells will be yours to enjoy, Wilbur-this lovely world, these precious days.'
     
    
# create a ToneAnalyzerV3 object, version 2017-09-21 using api key and url from config
authenticator = IAMAuthenticator(apikey=get_secret('APIKEY'))
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)
tone_analyzer.set_service_url(get_secret('URL'))

# get the text for analysis from the file
text = get_text_for_analysis()
if text:
    tone_analysis = tone_analyzer.tone(
        {'text': text},
        content_type='application/json'
    ).get_result()    
    print(tone_analysis)
else:
    print("No data")

# WORKING WITH THE OUTPUT OF TEXT ANALYSIS

### Watson Tone Analyser API Output
---

This is the output that the given text will produce. You will need to assign this output to a variable called **tone_analysis** in the function you are going to write below. 

```
{'document_tone': {'tones': [{'score': 0.582191, 'tone_id': 'sadness', 'tone_name': 'Sadness'}, {'score': 0.829888, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]}, 'sentences_tone': [{'sentence_id': 0, 'text': 'Team, I know that times are tough!', 'tones': [{'score': 0.801827, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]}, {'sentence_id': 1, 'text': 'Product sales have been disappointing for the past three quarters.', 'tones': [{'score': 0.817406, 'tone_id': 'sadness', 'tone_name': 'Sadness'}, {'score': 0.687768, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]}, {'sentence_id': 2, 'text': 'We have a competitive product, but we need to do a better job of selling it!', 'tones': [{'score': 0.506763, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]}]}

```



### Create (on paper) an idea of how this data might be organised into a data table

1.  How many bits of information are there about the document as a whole?
2.  How many bits of information are there about each sentence?
3.  If all tone analysis records were included in the dataframe, how many rows would there be?
4.  What information would be included in each row?

### Create a dataframe and start to populate with the data

You can create a **dataframe** from this data either by converting it manually into a table OR by using the pandas function pd.json_normalise(data).  

**Manually**:

One way to do this would be to create a list of dictionary records, with each record formed from the data from each row in the original 'sentences_tone' data.  You will need to loop through the rows in the 'sentences_tone' list, nesting a loop through the 'tones' list for each sentence.  For each, copy across the columns you feel should be included.

_Hint:_  
```
 for row in sentence_data:
        for col in row['tones']:
            new_row = {'sentence_id':row['sentence_id'], 'text':row['text'], 'tone_score':col['score'], 'tone_id':col['tone_id'],'tone_name':col['tone_name']}
```
**Using pandas**:

An alternative way to do this would be to create a pandas dataframe from the sentences_tone data list (using `pd.json_normalise(data)`)

In [21]:
import pandas as pd
import numpy as np

# convert json data to a dataframe with one row for each tone for each sentence
def convert_to_tones_table(data):
    # return the data normalized into a dataframe (pd.json_normalise(data))
    # the dataframe should have the columns: sentence_id, text, score, tone_id, tone_name
  tone_df = pd.json_normalize(data)
  return tone_df

tone_analysis = {"document_tone": {"tones": [{"score": 0.582191, "tone_id": "sadness", "tone_name": "Sadness"}, {"score": 0.829888, "tone_id": "analytical", "tone_name": "Analytical"}]}, "sentences_tone": [{"sentence_id": 0, "text": "Team, I know that times are tough!", "tones": [{"score": 0.801827, "tone_id": "analytical", "tone_name": "Analytical"}]}, {"sentence_id": 1, "text": "Product sales have been disappointing for the past three quarters.", "tones": [{"score": 0.817406, "tone_id": "sadness", "tone_name": "Sadness"}, {"score": 0.687768, "tone_id": "analytical", "tone_name": "Analytical"}]}, {"sentence_id": 2, "text": "We have a competitive product, but we need to do a better job of selling it!", "tones": [{"score": 0.506763, "tone_id": "analytical", "tone_name": "Analytical"}]}]}
tone_data = convert_to_tones_table(tone_analysis)

sen_data = tone_data['sentences_tone']
tone_table = pd.DataFrame.from_dict({})
for row in sen_data:
  for col in row:
    for rec in col['tones']:
      tone_table = pd.concat([tone_table, pd.DataFrame.from_records(rec, index=[0])], ignore_index=True)

display(tone_table)

,score,tone_id,tone_name
0,0.801827,analytical,Analytical
1,0.817406,sadness,Sadness
2,0.687768,analytical,Analytical
3,0.506763,analytical,Analytical


In [36]:
# Or by using the lamda function
import pandas as pd
import numpy as np

# convert json data to a dataframe with one row for each tone for each sentence
def convert_to_sentence_tones(data):
    df = pd.json_normalize(data)
    df = df.explode('tones', ignore_index=True)
    df['score'] = df['tones'].apply(lambda x: x['score'])
    df['tone_id'] = df['tones'].apply(lambda x: x['tone_id'])
    df['tone_name'] = df['tones'].apply(lambda x: x['tone_name'])
    df = df.drop(['tones'], axis=1)
    return df

tone_analysis = {"document_tone": {"tones": [{"score": 0.582191, "tone_id": "sadness", "tone_name": "Sadness"}, {"score": 0.829888, "tone_id": "analytical", "tone_name": "Analytical"}]}, "sentences_tone": [{"sentence_id": 0, "text": "Team, I know that times are tough!", "tones": [{"score": 0.801827, "tone_id": "analytical", "tone_name": "Analytical"}]}, {"sentence_id": 1, "text": "Product sales have been disappointing for the past three quarters.", "tones": [{"score": 0.817406, "tone_id": "sadness", "tone_name": "Sadness"}, {"score": 0.687768, "tone_id": "analytical", "tone_name": "Analytical"}]}, {"sentence_id": 2, "text": "We have a competitive product, but we need to do a better job of selling it!", "tones": [{"score": 0.506763, "tone_id": "analytical", "tone_name": "Analytical"}]}]}

#tone_analysis = {"document_tone": {"tones": [{"score": 0.525587, "tone_id": "sadness", "tone_name": "Sadness"}, {"score": 0.670614, "tone_id": "joy", "tone_name": "Joy"}, {"score": 0.802229, "tone_id": "confident", "tone_name": "Confident"}]}, "sentences_tone": [{"sentence_id": 0, "text": "But I feel peaceful.", "tones": [{"score": 0.511185, "tone_id": "joy", "tone_name": "Joy"}, {"score": 0.88939, "tone_id": "tentative", "tone_name": "Tentative"}]}, {"sentence_id": 1, "text": "Your success in the ring this morning was, to a small degree, my success.", "tones": [{"score": 0.919911, "tone_id": "joy", "tone_name": "Joy"}]}, {"sentence_id": 2, "text": "Your future is assured.", "tones": [{"score": 0.97759, "tone_id": "confident", "tone_name": "Confident"}]}, {"sentence_id": 3, "text": "You will live, secure and safe, Wilbur.", "tones": [{"score": 0.801827, "tone_id": "analytical", "tone_name": "Analytical"}, {"score": 0.92125, "tone_id": "confident", "tone_name": "Confident"}]}, {"sentence_id": 4, "text": "Nothing can harm you now.", "tones": []}, {"sentence_id": 5, "text": "These autumn days will shorten and grow cold.", "tones": []}, {"sentence_id": 6, "text": "The leaves will shake loose from the trees and fall.", "tones": [{"score": 0.621679, "tone_id": "fear", "tone_name": "Fear"}]}, {"sentence_id": 7, "text": "Christmas will come, and the snows of winter.", "tones": [{"score": 0.614764, "tone_id": "joy", "tone_name": "Joy"}]}, {"sentence_id": 8, "text": "You will live to enjoy the beauty of the frozen world, for you mean a great deal to Zuckerman and he will not harm you, ever.", "tones": [{"score": 0.930779, "tone_id": "joy", "tone_name": "Joy"}]}, {"sentence_id": 9, "text": "Winter will pass, the days will lengthen, the ice will melt in the pasture pond.", "tones": [{"score": 0.654012, "tone_id": "sadness", "tone_name": "Sadness"}]}, {"sentence_id": 10, "text": "The song sparrow will return and sing, the frogs will awake, the warm wind will blow again.", "tones": [{"score": 0.600542, "tone_id": "joy", "tone_name": "Joy"}]}, {"sentence_id": 11, "text": "All these sights and sounds and smells will be yours to enjoy, Wilbur-this lovely world, these precious days.", "tones": [{"score": 0.939404, "tone_id": "joy", "tone_name": "Joy"}, {"score": 0.660207, "tone_id": "confident", "tone_name": "Confident"}]}]}

# create sentences dataframe
sentences_data = convert_to_sentence_tones(tone_analysis['sentences_tone'])
display(sentences_data)

,sentence_id,text,score,tone_id,tone_name
0,0,"Team, I know that times are tough!",0.801827,analytical,Analytical
1,1,Product sales have been disappointing for the ...,0.817406,sadness,Sadness
2,1,Product sales have been disappointing for the ...,0.687768,analytical,Analytical
3,2,"We have a competitive product, but we need to ...",0.506763,analytical,Analytical


### Summarise the sentence data
*  Which sentence is the most analytical?
*  which sentence is the least analytical?
*  what is the average analytical tone score for the sentences?
*  what do the analytical scores look like in a bar chart?

In [37]:
def show_stats(df, tone_name):
  tone_df = df[df['tone_name'] == tone_name]
  most =  tone_df[tone_df['score'] == tone_df['score'].max()]
  least = tone_df[tone_df['score'] == tone_df['score'].min()]
  avg = round(tone_df['score'].mean(), 2)
  print(f'The most {tone_name} sentence:')
  print('-'*60)
  display(most[['text', 'score', 'tone_name']])
  print('\n\n')
  print(f'The least {tone_name} sentence:')
  print('-'*60)
  display(least[['text', 'score', 'tone_name']])
  print('\n\n')
  print(f'Average {tone_name} tone score: {avg}')
  print('-'*60)
  print('\n')
  return

show_stats(sentences_data, 'Analytical')
#show_stats(sentences_data, 'Sadness')

The most Analytical sentence:
------------------------------------------------------------


,text,score,tone_name
0,"Team, I know that times are tough!",0.801827,Analytical





The least Analytical sentence:
------------------------------------------------------------


,text,score,tone_name
3,"We have a competitive product, but we need to ...",0.506763,Analytical





Average Analytical tone score: 0.67
------------------------------------------------------------




### Challenging:  Report the tone data for the whole document
---

Play with the data, create a dataframe for the document_tone, tones data

 ```pd.json_normalize(document_tone)```  

Display the document score for each of the tones in the analysis

In [38]:
import pandas as pd
import numpy as np

# convert json data to a dataframe with one row for each tone for each sentence
def convert_to_document_tones(data):
    df = pd.json_normalize(data)
    df = df.explode('tones', ignore_index=True)
    df['score'] = df['tones'].apply(lambda x: x['score'])
    df['tone_id'] = df['tones'].apply(lambda x: x['tone_id'])
    df['tone_name'] = df['tones'].apply(lambda x: x['tone_name'])
    df = df.drop(['tones'], axis=1)
    return df

# create sentences dataframe
documents_data = convert_to_document_tones(tone_analysis['document_tone'])
display(documents_data)

,score,tone_id,tone_name
0,0.582191,sadness,Sadness
1,0.829888,analytical,Analytical


### Change the text in the text file and analyse the new text.
---

Here is some alternative, happier text.  Replace the text in the text-for-analysis.txt file with the text below.  Then run the notebook cells again to see the results.

But I feel peaceful. Your success in the ring this morning was, to a small degree, my success. Your future is assured. You will live, secure and safe, Wilbur. Nothing can harm you now. These autumn days will shorten and grow cold. The leaves will shake loose from the trees and fall. Christmas will come, and the snows of winter. You will live to enjoy the beauty of the frozen world, for you mean a great deal to Zuckerman and he will not harm you, ever. Winter will pass, the days will lengthen, the ice will melt in the pasture pond. The song sparrow will return and sing, the frogs will awake, the warm wind will blow again. All these sights and sounds and smells will be yours to enjoy, Wilbur-this lovely world, these precious days.

### The result of this analysis is below:

```
{'document_tone': {'tones': [{'score': 0.525587, 'tone_id': 'sadness', 'tone_name': 'Sadness'}, {'score': 0.670614, 'tone_id': 'joy', 'tone_name': 'Joy'}, {'score': 0.802229, 'tone_id': 'confident', 'tone_name': 'Confident'}]}, 'sentences_tone': [{'sentence_id': 0, 'text': 'But I feel peaceful.', 'tones': [{'score': 0.511185, 'tone_id': 'joy', 'tone_name': 'Joy'}, {'score': 0.88939, 'tone_id': 'tentative', 'tone_name': 'Tentative'}]}, {'sentence_id': 1, 'text': 'Your success in the ring this morning was, to a small degree, my success.', 'tones': [{'score': 0.919911, 'tone_id': 'joy', 'tone_name': 'Joy'}]}, {'sentence_id': 2, 'text': 'Your future is assured.', 'tones': [{'score': 0.97759, 'tone_id': 'confident', 'tone_name': 'Confident'}]}, {'sentence_id': 3, 'text': 'You will live, secure and safe, Wilbur.', 'tones': [{'score': 0.801827, 'tone_id': 'analytical', 'tone_name': 'Analytical'}, {'score': 0.92125, 'tone_id': 'confident', 'tone_name': 'Confident'}]}, {'sentence_id': 4, 'text': 'Nothing can harm you now.', 'tones': []}, {'sentence_id': 5, 'text': 'These autumn days will shorten and grow cold.', 'tones': []}, {'sentence_id': 6, 'text': 'The leaves will shake loose from the trees and fall.', 'tones': [{'score': 0.621679, 'tone_id': 'fear', 'tone_name': 'Fear'}]}, {'sentence_id': 7, 'text': 'Christmas will come, and the snows of winter.', 'tones': [{'score': 0.614764, 'tone_id': 'joy', 'tone_name': 'Joy'}]}, {'sentence_id': 8, 'text': 'You will live to enjoy the beauty of the frozen world, for you mean a great deal to Zuckerman and he will not harm you, ever.', 'tones': [{'score': 0.930779, 'tone_id': 'joy', 'tone_name': 'Joy'}]}, {'sentence_id': 9, 'text': 'Winter will pass, the days will lengthen, the ice will melt in the pasture pond.', 'tones': [{'score': 0.654012, 'tone_id': 'sadness', 'tone_name': 'Sadness'}]}, {'sentence_id': 10, 'text': 'The song sparrow will return and sing, the frogs will awake, the warm wind will blow again.', 'tones': [{'score': 0.600542, 'tone_id': 'joy', 'tone_name': 'Joy'}]}, {'sentence_id': 11, 'text': 'All these sights and sounds and smells will be yours to enjoy, Wilbur-this lovely world, these precious days.', 'tones': [{'score': 0.939404, 'tone_id': 'joy', 'tone_name': 'Joy'}, {'score': 0.660207, 'tone_id': 'confident', 'tone_name': 'Confident'}]}]}
```